In [1]:
import pandas as pd
import boto3
import json
import datetime
import configparser
import os
import glob
import psycopg2

def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

## Get Configuration

In [2]:
config = configparser.ConfigParser()
config.read(['dwh.cfg', 'secrets.cfg'])

KEY    = config.get('AWS','KEY')
SECRET = config.get('AWS','SECRET')
REGION = config.get('AWS','REGION')

DB_NAME=config.get('CLUSTER','DB_NAME')
DB_PORT=config.get('CLUSTER','DB_PORT')
DB_USER=config.get('CLUSTER','DB_USER')
DB_PASSWORD=config.get('CLUSTER','DB_PASSWORD') 

## Get Bucket

In [ ]:
bucket_name = "udacity-dend"
s3 = boto3.resource(
    "s3", region_name="us-west-2",
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET
)
bucket = s3.Bucket(bucket_name)

### Download Log Data

In [ ]:
s3.meta.client.download_file(bucket_name, "log_json_path.json", "log_json_path.json")

for log_object in bucket.objects.filter(Prefix="log_data"):
    if log_object.key.endswith(".json") == False:
        continue
    os.makedirs(log_object.key[:log_object.key.rindex("/")], exist_ok=True)
    s3.meta.client.download_file(bucket_name, log_object.key, log_object.key)

print("log data downloaded")

### Download Song Data

In [ ]:
for song_object in bucket.objects.filter(Prefix="song_data"):
    if song_object.key.endswith(".json") == False:
        continue
    os.makedirs(song_object.key[:song_object.key.rindex("/")], exist_ok=True)
    s3.meta.client.download_file(bucket_name, song_object.key, song_object.key)
print("song data downloaded")

### staging events table

Reading all log data for analysis the data is filter by page = "NextSong" since that is the only data that is used in the Data Warehouse.

In [38]:
df_staging_events = pd.concat([
    pd.read_json(file, lines=True) for file in get_files("log_data")
])

#### Sample Data

In [43]:
df_staging_events['ts']

0      1541903636796
1      1541903770796
2      1541904034796
3      1541910841796
4      1541910973796
           ...      
353    1543092353796
354    1543092558796
355    1543097750796
356    1543097978796
357    1543103174796
Name: ts, Length: 8056, dtype: int64

In [3]:
df_staging_events.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


#### Check for null data

In [4]:
df_staging_events.isna().sum()

artist           1236
auth                0
firstName         286
gender            286
itemInSession       0
lastName          286
length           1236
level               0
location          286
method              0
page                0
registration      286
sessionId           0
song             1236
status              0
ts                  0
userAgent         286
userId              0
dtype: int64

#### Check Data Types

In [5]:
df_staging_events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8056 entries, 0 to 357
Data columns (total 18 columns):
artist           6820 non-null object
auth             8056 non-null object
firstName        7770 non-null object
gender           7770 non-null object
itemInSession    8056 non-null int64
lastName         7770 non-null object
length           6820 non-null float64
level            8056 non-null object
location         7770 non-null object
method           8056 non-null object
page             8056 non-null object
registration     7770 non-null float64
sessionId        8056 non-null int64
song             6820 non-null object
status           8056 non-null int64
ts               8056 non-null int64
userAgent        7770 non-null object
userId           8056 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 1.2+ MB


#### Check Field Lengths of strings

In [6]:
print(f"artist maximum length: {df_staging_events['artist'].str.len().max()}")
print(f"auth maximum length: {df_staging_events['auth'].str.len().max()}")
print(f"firstName maximum length: {df_staging_events['firstName'].str.len().max()}")
print(f"lastName maximum length: {df_staging_events['lastName'].str.len().max()}")
print(f"gender maximum length: {df_staging_events['gender'].str.len().max()}")
print(f"level maximum length: {df_staging_events['level'].str.len().max()}")
print(f"location maximum length: {df_staging_events['location'].str.len().max()}")
print(f"method maximum length: {df_staging_events['method'].str.len().max()}")
print(f"page maximum length: {df_staging_events['page'].str.len().max()}")
print(f"song maximum length: {df_staging_events['song'].str.len().max()}")
print(f"userAgent maximum length: {df_staging_events['userAgent'].str.len().max()}")

artist maximum length: 89.0
auth maximum length: 10
firstName maximum length: 10.0
lastName maximum length: 9.0
gender maximum length: 1.0
level maximum length: 4
location maximum length: 46.0
method maximum length: 3
page maximum length: 16
song maximum length: 151.0
userAgent maximum length: 139.0


### Find page that are related to playing songs

In [37]:
df_staging_events.groupby(["page"]).size()

page
About                 36
Downgrade             60
Error                  9
Help                  47
Home                 806
Login                 92
Logout                90
NextSong            6820
Save Settings         10
Settings              56
Submit Downgrade       1
Submit Upgrade         8
Upgrade               21
dtype: int64

#### Remove all non song playing records

In [7]:
df_staging_events = df_staging_events[df_staging_events["page"] == "NextSong"].copy()

### users dimension table

In [8]:
df_user = df_staging_events[["userId", "firstName", "lastName", "gender", "level"]].copy()
df_user.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free


#### Check for null data

In [9]:
df_user.isna().sum()

userId       0
firstName    0
lastName     0
gender       0
level        0
dtype: int64

#### Check Data Types

In [10]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6820 entries, 0 to 357
Data columns (total 5 columns):
userId       6820 non-null object
firstName    6820 non-null object
lastName     6820 non-null object
gender       6820 non-null object
level        6820 non-null object
dtypes: object(5)
memory usage: 319.7+ KB


#### Check for duplicates

In [11]:
df_user.groupby(["userId"]).size().sort_values(ascending=False)

userId
49    687
80    659
97    557
15    389
29    346
     ... 
35      1
45      1
69      1
89      1
48      1
Length: 129, dtype: int64

#### Check field lengths of strings

In [12]:
print(f"firstName maximum length: {df_user['firstName'].str.len().max()}")
print(f"lastName maximum length: {df_user['lastName'].str.len().max()}")
print(f"gender maximum length: {df_user['gender'].str.len().max()}")
print(f"level maximum length: {df_user['level'].str.len().max()}")

firstName maximum length: 10
lastName maximum length: 9
gender maximum length: 1
level maximum length: 4


### time dimension table

In [13]:
df_time = df_staging_events[["ts"]].copy()
df_time.head()

,ts
0,1541903636796
1,1541903770796
2,1541904034796
4,1541910973796
5,1541911006796


#### Check for null data

In [14]:
df_time.isna().sum()

ts    0
dtype: int64

#### Check for duplicates

In [15]:
df_time.groupby(["ts"]).size().sort_values(ascending=False).head(5)

ts
1543339730796    2
1543422975796    2
1542984111796    2
1543069787796    2
1543435163796    2
dtype: int64

### songs staging table

In [16]:
df_song_staging = pd.concat([
    pd.read_json(file, lines=True) for file in get_files("song_data") if file.endswith(".json")
])

#### Sample Data

In [17]:
df_song_staging.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR052WY1187FB55B1C,27.49550,BRADENTON. FL,-82.57807,We The Kings,194.35057,1,SOGEKSP12A58A78530,August Is Over,2007
0,AR0S3YD1187B99967F,NaN,"Jacksonville, FL",NaN,The Classics IV,127.65995,1,SOYRACV12A8C13C4D8,24 Hours Of Loneliness,1975
0,ARNNRN31187B9AE7B7,NaN,"Forrest City, AR",NaN,Al Green,189.41342,1,SOIDBNB12A58A7A72C,Keep Me Crying,1976
0,ARZDP5D1187B9B4ECB,50.84838,Brussel 1981,4.34968,Arbeid Adelt!,279.03955,1,SOEDXTY12A6D4FB0ED,Het Meisje Van Mijn Hart,1983
0,ARFL99B1187B9A2A45,NaN,"PERTH AMBOY, New Jersey",NaN,Sugarland,202.26567,1,SOXTCXD12AB0183E39,Silent Night,2009


#### Check for null data

In [18]:
df_song_staging.isna().sum()

artist_id              0
artist_latitude     6789
artist_location        0
artist_longitude    6789
artist_name            0
duration               0
num_songs              0
song_id                0
title                  0
year                   0
dtype: int64

#### Check Data Types

In [19]:
df_song_staging.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10549 entries, 0 to 0
Data columns (total 10 columns):
artist_id           10549 non-null object
artist_latitude     3760 non-null float64
artist_location     10549 non-null object
artist_longitude    3760 non-null float64
artist_name         10549 non-null object
duration            10549 non-null float64
num_songs           10549 non-null int64
song_id             10549 non-null object
title               10549 non-null object
year                10549 non-null int64
dtypes: float64(3), int64(2), object(5)
memory usage: 906.6+ KB


#### Check Field Lengths of strings

In [20]:
print(f"artist_id maximum length: {df_song_staging['artist_id'].str.len().max()}")
print(f"artist_location maximum length: {df_song_staging['artist_location'].str.len().max()}")
print(f"artist_name maximum length: {df_song_staging['artist_name'].str.len().max()}")
print(f"song_id maximum length: {df_song_staging['song_id'].str.len().max()}")
print(f"title maximum length: {df_song_staging['title'].str.len().max()}")

artist_id maximum length: 18
artist_location maximum length: 176.0
artist_name maximum length: 117.0
song_id maximum length: 18
title maximum length: 173.0


### songs dimension table

In [21]:
df_song = df_song_staging[["song_id", "title", "artist_id", "year", "duration"]].copy()
df_song.head()

,song_id,title,artist_id,year,duration
0,SOGEKSP12A58A78530,August Is Over,AR052WY1187FB55B1C,2007,194.35057
0,SOYRACV12A8C13C4D8,24 Hours Of Loneliness,AR0S3YD1187B99967F,1975,127.65995
0,SOIDBNB12A58A7A72C,Keep Me Crying,ARNNRN31187B9AE7B7,1976,189.41342
0,SOEDXTY12A6D4FB0ED,Het Meisje Van Mijn Hart,ARZDP5D1187B9B4ECB,1983,279.03955
0,SOXTCXD12AB0183E39,Silent Night,ARFL99B1187B9A2A45,2009,202.26567


#### Check for null data

In [22]:
df_song.isna().sum()

song_id      0
title        0
artist_id    0
year         0
duration     0
dtype: int64

#### Check Data Types

In [23]:
df_song.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10549 entries, 0 to 0
Data columns (total 5 columns):
song_id      10549 non-null object
title        10549 non-null object
artist_id    10549 non-null object
year         10549 non-null int64
duration     10549 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 494.5+ KB


#### Check for duplicates

In [24]:
df_song.groupby(["song_id"]).size().sort_values(ascending=False)

song_id
SOZZZON12A8C139ED5    1
SOHZDYL12A8C141C73    1
SOHZCKI12AB018A082    1
SOHZBHI12AB017F7EE    1
SOHZAZP12A6D4F5CDE    1
                     ..
SOQTPLM12B0B809575    1
SOQTOBG12A6D4FD6F0    1
SOQTLMB12A58A7DBFF    1
SOQTKTG12A6701D451    1
SOAACTC12AB0186A20    1
Length: 10549, dtype: int64

#### Check field lengths of strings

In [25]:
print(f"artist_id maximum length: {df_song['artist_id'].str.len().max()}")
print(f"song_id maximum length: {df_song['song_id'].str.len().max()}")
print(f"title maximum length: {df_song['title'].str.len().max()}")

artist_id maximum length: 18
song_id maximum length: 18
title maximum length: 173.0


### artists dimension table

In [26]:
df_artist = df_song_staging[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].copy()
df_artist.head()


,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR052WY1187FB55B1C,We The Kings,BRADENTON. FL,27.49550,-82.57807
0,AR0S3YD1187B99967F,The Classics IV,"Jacksonville, FL",NaN,NaN
0,ARNNRN31187B9AE7B7,Al Green,"Forrest City, AR",NaN,NaN
0,ARZDP5D1187B9B4ECB,Arbeid Adelt!,Brussel 1981,50.84838,4.34968
0,ARFL99B1187B9A2A45,Sugarland,"PERTH AMBOY, New Jersey",NaN,NaN


#### Check for null data

In [27]:
df_artist.isna().sum()

artist_id              0
artist_name            0
artist_location        0
artist_latitude     6789
artist_longitude    6789
dtype: int64

#### Check Data Types

In [28]:
df_artist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10549 entries, 0 to 0
Data columns (total 5 columns):
artist_id           10549 non-null object
artist_name         10549 non-null object
artist_location     10549 non-null object
artist_latitude     3760 non-null float64
artist_longitude    3760 non-null float64
dtypes: float64(2), object(3)
memory usage: 494.5+ KB


#### Check for duplicates

In [29]:
df_artist.groupby(["artist_id"]).size().sort_values(ascending=False)

artist_id
ARUPWVD1187FB4DA36    9
ARYPTWE1187FB49D64    7
ARLHO5Z1187FB4C861    7
AR62BB21187B9AC83D    7
ARYF20K1187B9B76BD    7
                     ..
ARIRE5Q1187FB4DC13    1
ARUK0W21187B995456    1
ARIRLGQ1187FB45B7E    1
ARUJ5A41187FB3F5F1    1
AR00B1I1187FB433EB    1
Length: 7550, dtype: int64

#### Check field lengths of strings

In [30]:
print(f"artist_id maximum length: {df_artist['artist_id'].str.len().max()}")
print(f"artist_location maximum length: {df_artist['artist_location'].str.len().max()}")
print(f"artist_name maximum length: {df_artist['artist_name'].str.len().max()}")


artist_id maximum length: 18
artist_location maximum length: 176.0
artist_name maximum length: 117.0


### songplay fact table

In [31]:
df_songplay = df_staging_events[["ts", "userId", "level", "song", "artist", "sessionId", "location", "userAgent"]].copy()
df_songplay = pd.merge(df_songplay, df_artist, how='left', 
         left_on="artist", right_on="artist_name",
         left_index=False, right_index=False, copy=True)

df_songplay = df_songplay[["ts", "userId", "level", "song", "artist_id", "sessionId", "location", "userAgent"]].copy()
df_songplay = pd.merge(df_songplay, df_song, how='left', 
         left_on=["song", "artist_id"], right_on=["title", "artist_id"],
         left_index=False, right_index=False, copy=True)
df_songplay = df_songplay[["ts", "userId", "level", "song_id", "artist_id", "sessionId", "location", "userAgent"]].copy()
df_songplay.head()


,ts,userId,level,song_id,artist_id,sessionId,location,userAgent
0,1541903636796,69,free,NaN,NaN,455,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
1,1541903770796,69,free,NaN,NaN,455,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
2,1541904034796,69,free,NaN,AR3GZLR1187FB3D817,455,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
3,1541904034796,69,free,NaN,AR3GZLR1187FB3D817,455,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
4,1541910973796,32,free,NaN,NaN,456,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."


#### Check for null data

In [32]:
df_songplay.isna().sum()

ts              0
userId          0
level           0
song_id      9780
artist_id    3093
sessionId       0
location        0
userAgent       0
dtype: int64

#### Check Data Types

In [33]:
df_songplay.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10222 entries, 0 to 10221
Data columns (total 8 columns):
ts           10222 non-null int64
userId       10222 non-null object
level        10222 non-null object
song_id      442 non-null object
artist_id    7129 non-null object
sessionId    10222 non-null int64
location     10222 non-null object
userAgent    10222 non-null object
dtypes: int64(2), object(6)
memory usage: 718.7+ KB


#### Check for duplicates

In [34]:
df_songplay.fillna("", inplace=True)
df_songplay["all"] = (
    df_songplay["ts"].astype(str) + 
    "-" + df_songplay["userId"].astype(str) +
    "-" + df_songplay["level"].astype(str) +
    "-" + df_songplay["song_id"].astype(str) +
    "-" + df_songplay["artist_id"].astype(str) +
    "-" + df_songplay["sessionId"].astype(str) + 
    "-" + df_songplay["location"].astype(str) +
    "-" + df_songplay["userAgent"].astype(str)
)

df_songplay.groupby([
    "all"
]).size().sort_values(ascending=False).head(10)

all
1542669098796-25-paid--ARUPWVD1187FB4DA36-594-Marinette, WI-MI-"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"                                             9
1542165152796-80-paid--ARUPWVD1187FB4DA36-548-Portland-South Portland, ME-"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"                       9
1541668284796-80-paid--ARUPWVD1187FB4DA36-342-Portland-South Portland, ME-"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"                       9
1541183813796-71-free-SOBBZPM12AB017DF4B-ARH6W4X1187B99274F-70-Columbia, SC-"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D201 Safari/9537.53"    6
1542786780796-88-paid--ARH6W4X1187B99274F-744-Sacramento--Roseville--Arden-Arcade, CA-"Mozilla/5.0 (Macintosh; I

#### Check field lengths of strings

In [35]:
print(f"userId maximum length: {df_songplay['userId'].str.len().max()}")
print(f"level maximum length: {df_songplay['level'].str.len().max()}")
print(f"song_id maximum length: {df_songplay['song_id'].str.len().max()}")
print(f"artist_id maximum length: {df_songplay['artist_id'].str.len().max()}")
print(f"location maximum length: {df_songplay['location'].str.len().max()}")
print(f"userAgent maximum length: {df_songplay['userAgent'].str.len().max()}")

userId maximum length: 3.0
level maximum length: 4
song_id maximum length: 18
artist_id maximum length: 18
location maximum length: 46
userAgent maximum length: 139
